## Trading Algorithm Notes

### Pairs Trading
* If two securities are correlated, when they breakout from each other -> short the high one, buy the low one
* When prices move back together, close position
* Determine correlation using percent change and r-value

### Momentum Strategies
    * It is assumed price move will hold momentum for certain period or time
    * High volatility == Vix > 25
    * Use stop loss that follows the price upward and resets each time the stock price hits a new high
    * Additional functionality:
        1) Protect against downside moves with options trading
            * Buy put options on long positions (only buy 1) after running certain amount higher than previous high
            * Only buy when volatility is high
#### Gap and Go
    * Looks for overnight gappers and bets on continuation of that move
    * Open Short-term long trade and use stop loss
#### Earnings/News
    * Look at the most well known/liquid stocks that had earnings in the past few days
    * Focus on stocks that have an especially negative reaction from the market (-2%)
    * Open long position and use stop loss
#### Breakout Strategy
    * Look at past highs and generate buy signals as soon as price manages to break out from its previous high
    * Use dynamic lookback ranges based on volatility
        * When volatility is high, look further back
        * When low, look less
    * Open short-term long position and use stop loss

### Moving Avg Strategys
#### Look for rollovers
    * If faster MA crosses slower MA in upward motion -> buy
    * If faster MA crosses slower MA in downward motion -> sell
        * Consider adding a momentum threshold so that it doesnt trade too often
        * Momentum is identified by increasing distance between MA's
    * Where does 8-bar MA come in?
    * Decreases that trigger bearish moving average rollovers in multiple time frames offer short sale opportunity

#### Look for sideways markets - causes SMA's to become less accurate
    * Steady avg's
    * Frequent crossovers
    * High volatility, higher trading range    

#### Taking profits
    * Short sale - take profits when price lifts above 5-bar MA
    * Long sale - take profits when price dips below 5-bar MA
    
#### Exponential MA
    * multiplier = 2 ÷ (number of observations + 1)
    * EMA = Closing price x multiplier + EMA (previous day) x (1-multiplier)
    
#### MACD
* MACD=12-Period EMA − 26-Period EMA
* A 9-period EMA of the MACD called the "signal line," is then plotted on top of the MACD line as a trade indicator

## Creating Trading Robot

In [1]:
from config import Alpaca_key,Alpaca_secret_key,Alpaca_endpoint
import alpaca_trade_api as tradeapi
import pandas as pd

# API Auth
api = tradeapi.REST(Alpaca_key,Alpaca_secret_key,Alpaca_endpoint)

### Universe Selection Object

In [2]:
import time

def get_price_dataframe(ticker="SPY",length="5Min",limit=100): # length= 1Min, 5Min, 15Min, or 1D
    """
        Arguements:
            ticker -> Stock ticker symobl (default SPY)
            length -> Amount of time between bars (default 5Min)
            limit -> Number of bars returned (default 100)
            
        Returns:
        Returns a Pandas Dataframe of historical price data for any given ticker
    """
    # Submitting request
    barset=api.get_barset(ticker,length,limit)
    # Creating Dataframe
    data = pd.DataFrame()
    for bar in barset[ticker]:
        close=bar.c
        open=bar.o
        high=bar.h
        low=bar.l
        volume=bar.v
        datetime=bar.t
        if len(data) == 0:
            index = [0]
        else:
            index = [max(data.index)+1]
        new_row = pd.DataFrame({
            "date":datetime,
            "open":open,
            "high":high,
            "low":low,
            "close":close,
            "volume":volume}, index=index)
        data = data.append(new_row)
    return data

def tradeable_technology_stocks():
    """
        Returns:
            List of technology sector ticker symbols that meet filter criteria

        Notes:
            Tickers are gathered from csv files of all Technology stocks that trade on Nasdaq and S&P 500 and meet filter criteria
    """
    # Get Alpaca's stock list
    assets = api.list_assets("active", "us_equity")
    # Filter for TRADEABLE, SHORTABLE on Alapaca's platform
    assets = [asset for asset in assets if asset.tradable == True and asset.shortable == True]
    alpaca_tradable_symbols = [asset.symbol for asset in assets]
    # Filter for US EQUITY, TECHNOLOGY SECTOR, $2-$200 BILLION MARKET CAP, NASDAQ/S&P 500
    SP_tech_stocks = pd.read_csv("S&P_500_tech_assets_barchart_dotcom.csv")
    nasdaq_tech_stocks = pd.read_csv("nasdaq_tech_assets_nasdaq_dotcom.csv")
    combined_tickers = pd.concat([SP_tech_stocks["Symbol"], nasdaq_tech_stocks["Symbol"]], ignore_index=True)
    assets = [ticker for ticker in combined_tickers if ticker in alpaca_tradable_symbols]
    # Returns list of ticker symbols
    return assets

def find_most_liquid_stocks(periods=10, num_returned=10, bar_frequency="1D"):
    """
        Arguements:
            periods -> Number of prior days to use in avg volume calculation
                 -> High Volatility: Look back further, Low Volatility: Look back less
            N -> Number of ticker symbols to return
            
        Returns:
            List of the most liquid ticker symbols that meet below criteria:
                1) US Equity
                2) In the Technology Sector
                3) Trades on S&P 500 or Nasdaq Exchange
                4) $2-$200 Billion Market Cap
                5) Tradeable and Shortable on Alpaca's trading platform
    """
    # Returns list of technology sector ticker symbols that meet filter criteria
    tradable_stocks = tradeable_technology_stocks()
    
    # Finding the most liquid stocks by $ volume
    stock_volumes = {}
    count = 1
    for ticker in tradable_stocks:
        # Get prior 10 days of data
        data = get_price_dataframe(ticker, bar_frequency, periods)
        ################################# ADD OTHER FILTERS HERE ############################
        # Add avg volume to dictionary
        avg_10_day_volume = data["volume"].mean()
        stock_volumes[ticker] = avg_10_day_volume
        count+=1
        # Alpaca API limit is 200 calls per minute
        if count == 180:
            time.sleep(60)
    # Sort the dictionary, and return the N most liquid stocks 
    return [(ticker) for ticker, volume in sorted(stock_volumes.items(), key=lambda item: item[1])][-num_returned:]

### Alpha Object

In [3]:
import numpy as np

# Identifying 5 bar fractals (fractals remove noise of market swings to better identify key levels)
    # A fractal is a candlestick pattern made by 5 candles. 
    # The third candle has the lowest low price, the previous candles have decreasing lows and the next candles have increasing lows.
    # By this pattern, the low of the third candle is the support level.
    # The same concept applies to resistance levels, where the third candle has the highest high of the five ones.
def isSupport(df,i):
    support = df['low'][i] < df['low'][i-1] and df['low'][i] < df['low'][i+1] and df['low'][i+1] < df['low'][i+2] and df['low'][i-1] < df['low'][i-2]
    return support
def isResistance(df,i):
    resistance = df['high'][i] > df['high'][i-1]  and df['high'][i] > df['high'][i+1] and df['high'][i+1] > df['high'][i+2] and df['high'][i-1] > df['high'][i-2]
    return resistance

# Given a price value this function returns False if it is near a previously discovered key level.
# A level is near a previously discovered key level if their distance is less than the average candle size (s) in our chart
def isFarFromLevel(l, levels, s):
    levels = [(index,level) for index,level,category in levels]
    return np.sum([abs(l-x) < s  for x in levels]) == 0

def get_key_levels(tickers_to_trade, lookback_periods = 100, bar_frequency = "1D"):
    """
        Arguements:
            stocks -> List of tickers to find support levels for
            lookback_periods -> Number of bars to use when identifying key support levels
            bar_frequency -> Frequency of candles ('5Min', '1D', etc.. Consult Alpaca API )
    """
    key_levels = []

    for ticker in tickers_to_trade:
        df = get_price_dataframe(ticker, bar_frequency, lookback_periods)
        # Getting messy levels, will filter later
        levels = []
        for i in range(2,len(df)-2):
            if isSupport(df,i):
                levels.append((i, df['low'][i], "support"))
            elif isResistance(df,i):
                levels.append((i ,df['high'][i], "resistance"))

        # Average size of candlestick -> using later to check if already identified key levels that are similar
        s =  np.mean(df['high'] - df['low'])

        # Filtering levels to only unique levels
        levels = []
        for i in range(2,len(df)-2):
            if isSupport(df,i):
                l = df['low'][i]
                # Filters out levels close to already discovered levels
                if isFarFromLevel(l, levels, s):
                    levels.append((i,l,"support"))
            elif isResistance(df,i):
                l = df['high'][i]
                # Filters out levels close to already discovered levels
                if isFarFromLevel(l, levels, s):
                    levels.append((i,l,"resistance"))

        support = [level for index, level, category in levels if category == "support"]
        resistance = [level for index, level, category in levels if category == "resistance"]

        # Adding to master list
        key_levels.append({"ticker": ticker, "support": support,"resistance": resistance})
    return key_levels

def find_stocks_at_key_level(stocks_to_trade, proximity_threshold_percent = 2):
    """
        Arguments:
            stocks_to_trade -> List of stock elements. Each element is a dictionary with keys ('ticker': str, 'support': list, 'resistance': list)
            proximity_threshold_percent -> percentage difference used to judge if current price is close to support level
        
        Returns:
            stocks_at_support -> List of stock elements whose current price is within proximity_threshold_percent of support level
            stocks_at_resistance -> List of stock elements whose current price is within proximity_threshold_percent of resistance level
    """
    # Find if current price is near any of these resistance levels
    stocks_at_support = []
    stocks_at_resistance = []

    for stock in stocks_to_trade:
        # Get current price
        quote = api.get_last_quote(stock["ticker"])
        current_price = np.mean([quote.bidprice, quote.askprice])

        # Check if stock is near a support level
        for level in stock["support"]:
            difference = ((current_price - level)/level)*100
            # If stock is within 2% of a support level...
            if difference < proximity_threshold_percent and difference > -proximity_threshold_percent:
                near_support = True
            else:
                near_support = False

        # Check if stock is near a resistance level
        for level in stock["resistance"]:
            difference = ((current_price - level)/level)*100
            # If stock is within 2% of a support level...
            if difference < proximity_threshold_percent and difference > -proximity_threshold_percent:
                near_resistance = True
            else:
                near_resistance = False

        # Adding stocks near a key level to watch list
        if near_support:
            stocks_at_support.append(stock)
        elif near_resistance:
            stocks_at_resistance.append(stock)
            
    return stocks_at_support, stocks_at_resistance

### Portfolio Object

In [4]:
def is_market_open():
    clock = api.get_clock()
    if clock.is_open:
        return True
    else:
        return False

def is_account_blocked():
    account = api.get_account()
    if account.trading_blocked:
        return True
    else:
        return False
    
def is_ticker_owned_already(ticker):
    positions = api.list_positions()
    for position in positions:
        if position.symbol == ticker:
            qty = int(position.qty)
            if qty > 0:
                return True, qty
            elif qty < 0:
                return True, abs(qty)
    return False, 0

def get_buying_power():
    account = api.get_account()
    buying_power = account.buying_power
    return buying_power

def any_open_positions():
    positions = api.list_positions()
    if positions:
        return True     
    else:
        return False

### Transaction Cost Object

### Risk Object

### Execution Object

In [5]:
import random

def buy_stock(symbol='SPY',qty=1,type='market',time_in_force='gtc'):
    order_number = str(random.randint(100000000000,999999999999))
    api.submit_order(
        symbol=symbol,
        qty=qty,
        side='buy',
        type=type,
        time_in_force=time_in_force,
        client_order_id=order_number)
    return order_number

def sell_stock(symbol='SPY',qty=1,type='market',time_in_force='gtc'):
    order_number = str(random.randint(100000000000,999999999999))
    api.submit_order(
        symbol=symbol,
        qty=qty,
        side='sell',
        type=type,
        time_in_force=time_in_force,
        client_order_id=order_number)
    return order_number

## Current Strategy: Momentum
#### Breakout
    * Look at past highs and generate buy signals as soon as price manages to break out from its previous high
    * Use dynamic lookback ranges based on volatility
        * When volatility is high, look further back
        * When low, look less
    * Open short-term long position and use stop loss
#### Gap and Go
    * Looks for overnight gappers and bets on continuation of that move
    * Open Short-term long trade and use stop loss
#### Earnings/News
    * Look at the most well known/liquid stocks that had earnings in the past few days
    * Focus on stocks that have an especially negative reaction from the market (-2%)
    * Open long position and use stop loss

## Algorithm Structure
#### Universe Selection
    * Which asset class? (US Stocks)
    * Filters
        * Liquidity (Top 10 by dollar Volume)
        * Price (Too high or too low)
        * Industry (Tech Stocks)
#### Indicators
    * Calculate trade signal
    * Assign alpha scores to the securities that were passed from universe selection model
    * Example: Directional assumption of security and confidence in prediction
#### Portfolio 
    * Where trade signals from indicators are turned into a potential portfolio
    * Decides how much capital to allocate to each position
        * Same amount capital allocated to each trade signal
        * Can allocate more to a specific trade based upon higher levels of confidence if known
        * Doesnt allocate more than 75% of available funds
#### Transaction Cost
    * Account for transaction costs
    * Can be included in execution or portfolio model
    * Calculates
        * Commissions
        * Slippage
        * Market Impact
#### Risk
    * Monitors portfolios risk, manages positions when necessary
    * Example: Closes a position if down a certain percentage
    * If the position P/L > 5% or P/L < -2% close position
#### Execution
    * Order size, order type, order time, order price, etc.
    * Split large orders up into smaller orders over time?
    * Market, limit or other order type
    * Called every monday @ 10:00am
        * Opens or closes positions based on information it recieves from portfolio and risk model    

### Use This Area To Test Algorithm Improvements

In [13]:
# Finding the most liquid stocks that fit our criteria
stocks_to_trade_long = find_most_liquid_stocks(periods=10, num_returned=100, bar_frequency="1D")

# Calculating key levels of stocks to evaluate which stocks to trade
key_levels = get_key_levels(stocks_to_trade_long, lookback_periods=100, bar_frequency='1D')

# Filter out all stocks without calculated support or resistance levels
stocks_to_trade = [stock for stock in key_levels if stock["support"] and stock["resistance"]]

# Find stocks currently near a key level
stocks_at_support, stocks_at_resistance = find_stocks_at_key_level(stocks_to_trade, proximity_threshold_percent=1)

In [14]:
# proximity_threshold_percent = 1
len(stocks_at_support)

2

In [15]:
# proximity_threshold_percent = 1
len(stocks_at_resistance)

17

In [10]:
# proximity_threshold_percent = 2
len(stocks_at_support)

4

In [11]:
# proximity_threshold_percent = 2
len(stocks_at_resistance)

27

### Next Milestone in Project

In [9]:
# 1) For stocks to watch
    # If stock at resistance
        # Watch for breakout to the upside, if significant upward move -> BUY w/ stop loss until next resistance level
        # Watch for breakout to the downside -> SHORT w/ stop loss until closest support level

    # If stock at support
        # Watch for upside move, if significant upward move -> BUY w/ stop loss until next resistance level
        # Watch for breakout to the downside -> SHORT w/ stop loss until closest support level

# 2) For current positions
    # Find if stock is near a key level
    # If position at resistance
        # Watch for breakout to the upside, if significant upward move -> BUY w/ stop loss until next resistance level
        # Watch for breakout to the downside -> SHORT w/ stop loss until closest support level

    # If position at support
        # Watch for upside move, if significant upward move -> BUY w/ stop loss until next resistance level
        # Watch for breakout to the downside -> SHORT w/ stop loss until closest support level
    
    # If neither, hold?